In [2]:
import os
import pandas as pd
import numpy as np

os.chdir('C:/Users/fando/Box Sync/Springboard/Capstone Project/Data')

In [3]:
dtype_dict_ready = np.load('dtype_dict_ready.npy').item()
allData = pd.read_csv('allData_ready.txt', sep=" ", dtype = dtype_dict_ready, encoding = "ISO-8859-1")

In [4]:
allData.head()

,accommodates,bathrooms,bedrooms,beds,calculated_host_listings_count,extra_people,guests_included,host_has_profile_pic,host_identity_verified,host_is_superhost,...,summary_len,holiday,host_for,book_month,book_year,book_day,book_weekday,weekend,book_week,target
0,4,2.0,3.0,3.0,16,50.0,4,True,False,False,...,192,False,64,9,2015,1,1,False,36,1
1,4,2.0,3.0,3.0,16,50.0,4,True,False,False,...,192,False,64,9,2015,2,2,False,36,1
2,4,2.0,3.0,3.0,16,50.0,4,True,False,False,...,192,False,64,9,2015,3,3,False,36,1
3,4,2.0,3.0,3.0,16,50.0,4,True,False,False,...,192,False,64,9,2015,4,4,True,36,1
4,4,2.0,3.0,3.0,16,50.0,4,True,False,False,...,192,True,64,9,2015,5,5,True,36,1


In [17]:
allData.drop(['book_day'], axis=1, inplace=True)

In [18]:
X = allData.drop('target', axis=1)
y = allData['target']

Split the data by quarter

In [4]:
Q1 = allData.loc[(allData['book_month'].isin(['1','2','3']))]
Q2 = allData.loc[(allData['book_month'].isin(['4','5','6']))]
Q3 = allData.loc[(allData['book_month'].isin(['7','8','9']))]
Q4 = allData.loc[(allData['book_month'].isin(['10','11','12']))]

In [5]:
X_Q1 = Q1.drop('target', axis=1)
y_Q1 = Q1['target']
X_Q2 = Q2.drop('target', axis=1)
y_Q2 = Q2['target']
X_Q3 = Q3.drop('target', axis=1)
y_Q3 = Q3['target']
X_Q4 = Q4.drop('target', axis=1)
y_Q4 = Q4['target']

## Fit model by Quarter

### Random Forest

Q1 is much harder to fit.

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

In [14]:
cross_val_score(RandomForestClassifier(max_depth=20, n_estimators = 75, n_jobs=-1), X_Q1, y_Q1, scoring='roc_auc', cv=4).mean()

0.69663831082190275

In [15]:
cross_val_score(RandomForestClassifier(max_depth=20, n_estimators = 75, n_jobs=-1), X_Q2, y_Q2, scoring='roc_auc', cv=4).mean()

0.86898545315607501

In [16]:
cross_val_score(RandomForestClassifier(max_depth=20, n_estimators = 75, n_jobs=-1), X_Q3, y_Q3, scoring='roc_auc', cv=4).mean()

0.89399549357035535

In [17]:
cross_val_score(RandomForestClassifier(max_depth=20, n_estimators = 75, n_jobs=-1), X_Q4, y_Q4, scoring='roc_auc', cv=4).mean()

0.84913860956863074

### Neural Network

Roughly equal performances on Q1-Q4. Better fit for Q1 than Random Forest.

In [19]:
from sklearn.neural_network import MLPClassifier

In [21]:
cross_val_score(MLPClassifier(activation='logistic', solver = 'adam', hidden_layer_sizes=260), X_Q1, y_Q1, scoring='roc_auc', cv=4).mean()

0.78446441393438615

In [22]:
cross_val_score(MLPClassifier(activation='logistic', solver = 'adam', hidden_layer_sizes=260), X_Q2, y_Q2, scoring='roc_auc', cv=4).mean()

0.79741370116974675

In [23]:
cross_val_score(MLPClassifier(activation='logistic', solver = 'adam', hidden_layer_sizes=260), X_Q3, y_Q3, scoring='roc_auc', cv=4).mean()

0.79376017481025563

In [24]:
cross_val_score(MLPClassifier(activation='logistic', solver = 'adam', hidden_layer_sizes=260), X_Q4, y_Q4, scoring='roc_auc', cv=4).mean()

0.79291479295768053

### Gradient Boost

Similar story. Q1 is much harder to fit.

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

In [26]:
cross_val_score(GradientBoostingClassifier(n_estimators = 80, max_depth = 20), X_Q1, y_Q1, scoring='roc_auc', cv=4).mean()

0.72310949464723351

In [34]:
cross_val_score(GradientBoostingClassifier(n_estimators = 80, max_depth = 20), X_Q2, y_Q2, scoring='roc_auc', cv=4).mean()

0.93181635512808181

In [35]:
cross_val_score(GradientBoostingClassifier(n_estimators = 80, max_depth = 20), X_Q3, y_Q3, scoring='roc_auc', cv=4).mean()

0.94802014732786744

In [36]:
cross_val_score(GradientBoostingClassifier(n_estimators = 80, max_depth = 20), X_Q4, y_Q4, scoring='roc_auc', cv=4).mean()

0.8847422153170591

## Ensemble by Quarter

It seems like the unpredictability of Q1 drags down the accuracy of Random Forest and Gradient Boosting when they are fitted on the whole dataset. Let's fit Q1 and Q2-4 separately.

In [7]:
from sklearn.cross_validation import train_test_split
from sklearn import metrics

In [8]:
Q2_4 = allData.loc[~allData['book_month'].isin(['1','2','3'])]
X_Q24 = Q2_4.drop('target', axis=1)
y_Q24 = Q2_4['target']

In [51]:
X_trainQ1, X_testQ1, y_trainQ1, y_testQ1 = train_test_split(X_Q1, y_Q1, test_size=0.2, random_state=0)
X_trainQ24, X_testQ24, y_trainQ24, y_testQ24 = train_test_split(X_Q24, y_Q24, test_size=0.2, random_state=0)

In [40]:
# Q1: Neural Net
clf1 = MLPClassifier(activation='logistic', solver = 'adam', hidden_layer_sizes=260)
clf1.fit(X_trainQ1, y_trainQ1)
y_predQ1 = clf1.predict(X_testQ1)

In [52]:
# Q2-Q4: Gradient Boosting
clf2 = GradientBoostingClassifier(n_estimators = 80, max_depth = 20)
clf2.fit(X_trainQ24, y_trainQ24)
y_predQ24 = clf2.predict(X_testQ24)

In [53]:
y_pred = np.append(y_predQ1, y_predQ24)

In [57]:
y_test = np.append(y_testQ1, y_testQ24)

In [58]:
metrics.accuracy_score(y_test, y_pred)

0.94859715242881071

If fit Gradient Boosting on the whole dataset using cv=4, accuracy=0.88. Fitting Q1 using a different model improves the accuracy greatly.

### However, the Winner goes to XGBoost

Still, Q1 is the most unpredictable. However, out of all the models we tried, XGBoost gave the best results on both Q1 and Q2~Q4. 

We've showed that fitting different models on different quarters is helpful. Yet it's still not powerful enough to beat XGBoost. 

In [1]:
import xgboost as xgb

In [ ]:
xgb.XGBClassifier(n_estimators= 80, max_depth= 20, objective= 'binary:logistic', nthread= -1)

In [10]:
def to_int(df, var):
    df[var] = df[var].astype(int)
    return df

In [20]:
for var in ['book_month', 'book_year', 'book_weekday']:
    X_Q24 = to_int(X_Q24, var)
    X_Q1 = to_int(X_Q1, var)
    X = to_int(X, var)

In [23]:
cross_val_score(xgb.XGBClassifier(n_estimators= 160, max_depth= 10, objective= 'binary:logistic', n_jobs= -1), X_Q24, y_Q24, scoring='roc_auc', cv=4).mean()

0.96290783101250721

In [24]:
cross_val_score(xgb.XGBClassifier(n_estimators= 160, max_depth= 10, objective= 'binary:logistic', n_jobs= -1), X_Q1, y_Q1, scoring='roc_auc', cv=4).mean()

0.91807952122772363

In [25]:
cross_val_score(xgb.XGBClassifier(n_estimators= 160, max_depth= 10, objective= 'binary:logistic', n_jobs= -1), X, y, scoring='roc_auc', cv=4).mean()

0.95851595929893896